In [1]:
import numpy as np
import random
from numba import jit, prange

In [2]:
# simulation codee
@jit(nopython=True, parallel=True, fastmath=True)
def simulate(sim_array, sims):
  for sim in prange(sims):
    sim_array[sim] = np.random.randint(1, 7) + np.random.randint(1, 7)
    # sim_array[sim] = random.randint(1, 6) + random.randint(1, 6)

In [3]:
%%time
# configuration for the sim
num_rolls = 10
num_sims = 1000 * 1000 
num_results = 1000
# create array to store the sim
sim_array = np.zeros((num_sims, num_rolls), np.int8).reshape((num_rolls * num_sims,))
# do the simulation
simulate(sim_array, num_sims * num_rolls)

CPU times: user 624 ms, sys: 76.9 ms, total: 701 ms
Wall time: 604 ms


In [4]:
%%time
sim_array = sim_array.reshape((num_sims, num_rolls))
sim_array.sort()
# get unique sets of rols and their counts
# the view in the next line only works if the array is C-contiguous
sim_array = np.ascontiguousarray(sim_array)
# np.unique() finds identical items in a raveled array. To make it
# see each row as a single item, we create a view of each row as a
# byte string of length itemsize times number of columns in `ar`
sim_array_row_view = sim_array.view('|S%d' % (sim_array.itemsize * sim_array.shape[1]))
_, unique_idxs, counts = np.unique(sim_array_row_view, axis=0, return_counts=True, return_index=True)
# sort the unique and counts desceending
sorted_unique_idxs = np.argsort(counts)[::-1]
sorted_unique = sim_array[unique_idxs][sorted_unique_idxs, :]
sorted_counts = counts[sorted_unique_idxs]
# concatenate for viewing
results = np.concatenate((sorted_unique, sorted_counts.reshape(-1, 1)), axis=1)

CPU times: user 1.53 s, sys: 34.4 ms, total: 1.57 s
Wall time: 1.17 s


In [5]:
for i in range(num_results):
  print(list(results[i]))

[4, 5, 6, 6, 7, 7, 8, 8, 9, 10, 380]
[4, 5, 6, 6, 7, 7, 8, 9, 9, 10, 339]
[4, 5, 6, 6, 7, 7, 7, 8, 9, 10, 338]
[4, 5, 6, 7, 7, 7, 8, 8, 9, 10, 331]
[3, 4, 5, 6, 7, 7, 8, 8, 9, 10, 325]
[4, 5, 5, 6, 7, 7, 8, 8, 9, 10, 324]
[4, 5, 6, 7, 7, 8, 8, 9, 9, 10, 324]
[4, 5, 6, 7, 7, 8, 8, 9, 10, 11, 318]
[4, 5, 6, 6, 7, 7, 8, 9, 10, 11, 309]
[4, 5, 5, 6, 6, 7, 7, 8, 9, 10, 309]
[3, 4, 5, 6, 6, 7, 7, 8, 9, 10, 307]
[3, 4, 5, 5, 6, 7, 7, 8, 9, 10, 284]
[4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 281]
[5, 6, 6, 7, 7, 7, 8, 8, 9, 10, 281]
[4, 5, 6, 7, 7, 7, 8, 9, 9, 10, 276]
[3, 5, 6, 6, 7, 7, 8, 8, 9, 10, 276]
[4, 5, 6, 7, 7, 8, 8, 8, 9, 10, 275]
[4, 5, 5, 6, 7, 7, 8, 9, 9, 10, 273]
[4, 5, 6, 6, 7, 8, 8, 9, 9, 10, 272]
[3, 4, 5, 6, 7, 7, 8, 9, 9, 10, 271]
[4, 5, 6, 6, 7, 7, 7, 8, 8, 9, 270]
[4, 5, 5, 6, 7, 7, 8, 9, 10, 11, 270]
[4, 5, 6, 6, 7, 7, 8, 8, 9, 9, 269]
[4, 5, 6, 6, 7, 8, 8, 9, 10, 11, 268]
[5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 267]
[3, 4, 5, 6, 6, 7, 7, 8, 8, 9, 265]
[4, 4, 5, 6, 7, 7, 8, 8, 9, 10, 259]
[

In [13]:
df

,count,sim,roll,roll_value
0,380,0,1,4
1,339,1,1,4
2,338,2,1,4
3,331,3,1,4
4,325,4,1,3
...,...,...,...,...
495,231,45,10,11
496,230,46,10,11
497,228,47,10,11
498,227,48,10,9


In [46]:
import altair as alt
import pandas as pd

In [68]:
pd.DataFrame(results[:50], columns=[i + 1 for i in range(num_rolls)] + ['count']).reset_index().rename(columns={"index": "sim"}).head()

,sim,1,2,3,4,5,6,7,8,9,10,count
0,0,4,5,6,6,7,7,8,8,9,10,380
1,1,4,5,6,6,7,7,8,9,9,10,339
2,2,4,5,6,6,7,7,7,8,9,10,338
3,3,4,5,6,7,7,7,8,8,9,10,331
4,4,3,4,5,6,7,7,8,8,9,10,325


In [47]:
df = pd.DataFrame(results[:50], columns=[i + 1 for i in range(num_rolls)] + ['count'])
df = df.reset_index().rename(columns={"index": "sim"}).melt(['count', 'sim'], \
    var_name='roll', value_name='roll_value')

In [57]:
df.loc[df['count'] == 380]

,count,sim,roll,roll_value
0,380,0,1,4
50,380,0,2,5
100,380,0,3,6
150,380,0,4,6
200,380,0,5,7
250,380,0,6,7
300,380,0,7,8
350,380,0,8,8
400,380,0,9,9
450,380,0,10,10


In [65]:
circle = alt.Chart(df).mark_circle().encode(
    x='sim:O',
    y='roll_value:O',
    size=alt.Size('count()', 
                  # scale=alt.Scale(type='log')
                  ),
    tooltip=['count()']
)

bar = alt.Chart(df).mark_bar().encode(
    x='sim:O',
    y=alt.Y('count:Q', 
            # scale=alt.Scale(type='log')
            ),
    tooltip=['count:Q']
)

circle & bar

alt.VConcatChart(...)